In [1]:
!pip install yfinance
import yfinance as yf   
import pandas as pd
import numpy as np

In [5]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC


In [6]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","BTC-USD","LTC-USD","TRX-USD","GGPI","BIM","AI","AMZN"]
symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD"]
#symbols = ["MSFT","AMZN","AAPL","TSLA","GOOG","GOOGL"]

dfs = []
for i in symbols:
   data = yf.download(i,period="60d",interval="5m")
   dfs.append(data)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [7]:
for df in dfs:
  print(df.isnull().sum(),end=" ")
  print(df.shape)

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64 (17157, 6)
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64 (17149, 6)
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64 (17151, 6)
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64 (17155, 6)
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64 (17157, 6)
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64 (17150, 6)
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64 (17158, 6)
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64 (17154, 6)
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       

In [8]:
fixed_dfs = []
for df in dfs:
    full = []
    df = np.array(df)
    for i in range(10, df.shape[0]-1):
        ten_days_ago = i-10
        nine_days_ago = i-9
        eight_days_ago = i-8
        seven_days_ago = i-7
        six_days_ago = i-6
        five_days_ago = i-5
        four_days_ago = i-4
        three_days_ago = i-3
        two_days_ago = i - 2
        yesterday = i - 1
        today = i
        tomorrow = i + 1
        if df[tomorrow][3] > df[today][3]:
            future = 1
        else:
            future = 0
        new_lst = [df[ten_days_ago][3], df[nine_days_ago][3], df[eight_days_ago][3], df[seven_days_ago][3], df[six_days_ago][3],
                   df[five_days_ago][3], df[four_days_ago][3], df[three_days_ago][3], df[two_days_ago][3], df[yesterday][3], df[today][3], future]
        full.append(new_lst)
    fuu = np.array([full])
    lst = []
    for i in range(0, fuu.shape[1]):
        lst.append(fuu[0][i])
    mm = np.array(lst)
    dff = pd.DataFrame(mm, columns=["ten_days_ago", "nine_days_ago", "eight_days_ago", "seven_days_ago", "six_days_ago",
                           "five_days_ago", "four_days_ago", "three_days_ago", "two_days_ago", "yesterday", "today", "result"])
    scaler = MinMaxScaler()
    scaled = pd.DataFrame(scaler.fit_transform(dff.T).T, columns=["ten_days_ago", "nine_days_ago", "eight_days_ago", "seven_days_ago",
                              "six_days_ago", "five_days_ago", "four_days_ago", "three_days_ago", "two_days_ago", "yesterday", "today", "result"])
    scaled.drop("result",axis=1,inplace=True)
    scaled["result"] = dff["result"]
    fixed_dfs.append(scaled)

In [9]:
for i in fixed_dfs:
  print(i.shape)

(17146, 12)
(17138, 12)
(17140, 12)
(17144, 12)
(17146, 12)
(17139, 12)
(17147, 12)
(17143, 12)
(17143, 12)
(17147, 12)
(17143, 12)


In [10]:
data = fixed_dfs[0]
for i in range(1,len(symbols)):
  data = pd.concat([data,fixed_dfs[i]], ignore_index = True)

In [11]:
data

,ten_days_ago,nine_days_ago,eight_days_ago,seven_days_ago,six_days_ago,five_days_ago,four_days_ago,three_days_ago,two_days_ago,yesterday,today,result
0,1.000000,0.997996,0.996235,0.993588,0.995643,0.996397,0.998308,0.997241,0.998524,0.999151,0.999291,0.0
1,0.998703,0.996941,0.994292,0.996348,0.997103,0.999016,0.997948,0.999232,0.999859,1.000000,0.998926,1.0
2,0.996941,0.994292,0.996348,0.997103,0.999016,0.997948,0.999232,0.999859,1.000000,0.998926,0.999748,1.0
3,0.993584,0.995639,0.996393,0.998304,0.997237,0.998520,0.999147,0.999288,0.998214,0.999036,1.000000,0.0
4,0.995639,0.996393,0.998304,0.997237,0.998520,0.999147,0.999288,0.998214,0.999036,1.000000,0.999860,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
188571,0.994076,0.992466,0.993121,0.992474,0.989642,0.991769,0.992144,0.992436,0.994775,1.000000,0.998033,0.0
188572,0.992069,0.992759,0.992078,0.989097,0.991336,0.991731,0.992038,0.994500,1.000000,0.997930,0.997315,1.0
188573,0.992759,0.992078,0.989097,0.991336,0.991731,0.992038,0.994500,1.000000,0.997930,0.997315,0.999515,1.0
188574,0.984613,0.981655,0.983877,0.984269,0.984574,0.987017,0.992476,0.990421,0.989811,0.991995,1.000000,1.0


In [12]:
data = data.dropna()

In [13]:
X = data.drop("result",axis=1)
xTrain, xTest, yTrain, yTest = train_test_split(X, data["result"], test_size=0.1,random_state=11)

print(xTrain.shape)
print(yTrain.shape)

print(xTest.shape)
print(yTest.shape)


(169718, 11)
(169718,)
(18858, 11)
(18858,)


In [131]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
clf.score(xTest,yTest)

0.6784255477163015

In [132]:
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
logisticRegr.score(xTest, yTest)

0.6687178399023924

In [ ]:
svm = SVC()
svm.fit(xTrain,yTrain)
svm.score(xTest,yTest)

In [ ]:
from sklearn.neighbors import NearestCentroid
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
nc.score(xTest,yTest)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(xTrain, yTrain).score(xTest,yTest)

In [ ]:
from sklearn import tree
tree = tree.DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
tree.score(xTest,yTest)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.score(xTest, yTest)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf2 = KNeighborsClassifier(n_neighbors=7)
clf2.fit(xTrain, yTrain)
clf2.score(xTest,yTest)

In [14]:
model = Sequential()
model.add(Dense(64, activation='relu', kernel_initializer='he_normal', input_shape=(11,)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu', kernel_initializer='he_normal', input_shape=(11,)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu', kernel_initializer='he_normal', input_shape=(11,)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                768       
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 3

In [15]:
model.fit(xTrain,yTrain,epochs=15,batch_size=32,validation_data=(xTest,yTest))

Epoch 1/15
5304/5304 [==============================] - 15s 3ms/step - loss: 0.5710 - accuracy: 0.6542 - val_loss: 0.5673 - val_accuracy: 0.6588
Epoch 2/15
5304/5304 [==============================] - 13s 2ms/step - loss: 0.5657 - accuracy: 0.6579 - val_loss: 0.5664 - val_accuracy: 0.6589
Epoch 3/15
5304/5304 [==============================] - 14s 3ms/step - loss: 0.5653 - accuracy: 0.6582 - val_loss: 0.5665 - val_accuracy: 0.6588
Epoch 4/15
5304/5304 [==============================] - 13s 3ms/step - loss: 0.5653 - accuracy: 0.6581 - val_loss: 0.5661 - val_accuracy: 0.6594
Epoch 5/15
5304/5304 [==============================] - 13s 3ms/step - loss: 0.5646 - accuracy: 0.6589 - val_loss: 0.5642 - val_accuracy: 0.6619
Epoch 6/15
5304/5304 [==============================] - 14s 3ms/step - loss: 0.5630 - accuracy: 0.6611 - val_loss: 0.5640 - val_accuracy: 0.6622
Epoch 7/15
5304/5304 [==============================] - 13s 2ms/step - loss: 0.5629 - accuracy: 0.6611 - val_loss: 0.5647 - val_ac